In [5]:
import numpy as np
import scipy.linalg as la
data = np.load('C:\\Users\\zihan\\Desktop\\Files\\Research\\bryankelly_without_missing.npz')
processed_data = data['processed_data']
T, N, M = processed_data.shape
chara_data_all = processed_data[:, :, 2:]
# print (processed_data[:,:,2])

In [8]:
chara_data_t= chara_data[0,:]
median0= np.median(chara_data_t[:,0])
index0= chara_data_t[:,0]<= median0
median1= np.median(chara_data_t[:,1])
index1= index0 & (chara_data_t[:,1]<= median1)
median2= np.median(chara_data_t[:,2])
index2= index1 & (chara_data_t[:,2]<= median2)
median3= np.median(chara_data_t[:,3])
index3= index2 & (chara_data_t[:,3]<= median3)
print (sum(index0), sum(index1), sum(index2), sum(index3))

152 61 46 10


In [28]:
import itertools
allcombination= list(itertools.combinations(range(chara_data_all.shape[2]), 38))
maximum= 0
returns= processed_data[:,:,1]
count= 0
record= []
for combination in allcombination:
    if count%10000==0:
        print ('We are here at %d'%(count))
    count+= 1
    chara_data = processed_data[:, :, combination]
    # print (chara_data.shape)
    for i in range(T):
        for j in range(chara_data.shape[2]):
            chara_data[i, :, j] = np.argsort(chara_data[i, :, j])
            chara_data[i, :, j] = (chara_data[i, :, j] + 1) / (N + 1)
# The dimension of characteristics: 
    portfolio = np.zeros((T - 1, chara_data.shape[2]))
    for i in range(T - 1):
        portfolio[i, :] = returns[i + 1, :].dot(chara_data[i, :, :])
    portfolio_demean = portfolio - np.mean(portfolio, axis=0)
# Dimension of portfolio: T*M
    covmat = portfolio_demean.T.dot(portfolio_demean) / (T - 2)
    variance, loading = np.linalg.eig(covmat)
    K= 3
    Factor = np.zeros((int(T / 2), K))
    start= int(T/2)-1
    for t in range(int(T / 2)):
        # if t%20==0:
            # print ('Have gone through iteration %d'%(t))
        portfolio_t = portfolio[:(t+start), :]
        portfolio_demean_t = portfolio_t - np.mean(portfolio_t, axis=0)
        covmat_t = portfolio_demean_t.T.dot(portfolio_demean_t) / (t+start - 1)
        variance_t, loading_t = la.eigh(covmat_t,eigvals= (0,K-1))
        Factor[t, :] = portfolio[t + start, :].dot(loading_t[:, :K])
    mu = np.mean(Factor[:, :K], axis=0)
    covmat_factor = np.cov(Factor[:, :K].T)
    output=(np.sqrt(np.dot(np.dot(mu, np.linalg.inv(covmat_factor)), mu.T)))
    if output>maximum:
        maximum= max(maximum, output)
        record= combination

We are here at 0


In [29]:
print (maximum)
print (combination)
# print (chara_data_all[:,:,2])

0.290621793234
(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39)
